In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback


Using TensorFlow backend.


In [2]:
# new_model = keras.models.load_model('ltsm_test.h5')
auc_scores = []

In [3]:
path = '../data/'
comp = 'kaggle ltsm/'
EMBEDDING_FILE=f'{path}{comp}glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv'
TEST_DATA_FILE=f'{path}{comp}test.csv'

In [4]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [5]:
test = pd.read_csv(TEST_DATA_FILE)
train = pd.read_csv(TRAIN_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)



In [16]:
print(list_sentences_test[0])
print(list_tokenized_test[0])
print(X_te[0])

Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,
[2665, 655, 8849, 656, 8, 57, 16388, 83, 884, 356, 16, 3222, 76, 21, 6, 4, 6865, 6, 1521, 7, 56, 655, 4942, 1898, 682, 6908, 4, 96, 6, 2, 5104, 29, 417, 6, 726, 35, 8849, 656, 8, 36, 4122, 10, 2818, 660, 437, 454, 19612, 9, 333, 15, 153, 4, 8, 240, 49, 52, 24, 5, 2045, 162, 3132, 682, 2880, 96, 219, 145, 493, 84]
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0  2665   655  8849   656
     8    57 16388    83   884   356    16  3222    76    21     6     4
  6865     6  1521     7 

In [ ]:
new_model = keras.models.load_model('ltsm_test.h5')

In [31]:
test_comment = ["It is impossible to talk to you. You are rude and uneducated. How on earth are you alive?"]
tokenized_test = tokenizer.texts_to_sequences(test_comment)
print(tokenized_test)
final_test = pad_sequences(tokenized_test, maxlen=maxlen)
print(final_test)



[[11, 8, 1915, 2, 38, 2, 6, 6, 19, 1707, 4, 7753, 70, 15, 1271, 19, 6, 3286]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   11    8
  1915    2   38    2    6    6   19 1707    4 7753   70   15 1271   19
     6 3286]]


In [32]:
y_test = new_model.predict([final_test], batch_size=1024, verbose=1)


1/1 [==============================] - 0s 19ms/step


In [33]:
print(y_test)

[[0.6838008  0.00395226 0.04134396 0.02384126 0.28275222 0.02817651]]


In [44]:
np.set_printoptions(precision=10, suppress=True)
print(len(np.where( y_test > 0.5 )))
# "{:.3f}".format(y_test[1])

2


In [47]:
result_classes = []

for item in y_test:
    result_classes.append(list_classes[np.argmax(item)])
    


["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
 '== From RfC == \n\n The title is fine as it is, IMO.'
 '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'
 ":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
 "I don't anonymously edit articles at all."
 'Thank you for understanding. I think very highly of you and would not revert without discussion.'
 'Please do not add nonsense to Wikipedia. Such edits are considered vandalism and quickly undone. If you would like to experiment, please use the sandbox in

In [51]:
# print(list_sentences_test[:10])
# print("msg: {0} class: {1}".format(list_sentences_test[:10], result_classes[:10]))

msg: ["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
 '== From RfC == \n\n The title is fine as it is, IMO.'
 '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'
 ":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
 "I don't anonymously edit articles at all."
 'Thank you for understanding. I think very highly of you and would not revert without discussion.'
 'Please do not add nonsense to Wikipedia. Such edits are considered vandalism and quickly undone. If you would like to experiment, please use the sandb

In [57]:
df_tr = pd.read_csv(TRAIN_DATA_FILE)
df_t = pd.read_csv(TEST_DATA_FILE)



,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [ ]:
df_t.head(20)